# Импорт

In [3]:
import pandas as pd
from openai import OpenAI

ModuleNotFoundError: No module named 'openai'

# Датасет

In [4]:
file_path = 'LK_modified.xlsx'
sheet_name = 'Вопрос ответ'

def save_to_excel(df, file_path, sheet_name):
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df[['question', 'content', 'category']].to_excel(writer, sheet_name=sheet_name, index=False)

In [5]:
df = pd.read_excel(file_path, sheet_name = 'Вопрос ответ', index_col='Unnamed: 0')

In [6]:
df

,question,content,category
0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль
1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль
2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль
3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль
4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль
...,...,...,...
1702,Ошибка 500,Данная ошибка возможна при переходе на портал\...,обучение
1703,при входе на портал обучения выходит ошибка 505,Данная ошибка возможна при переходе на портал\...,обучение
1704,привязать другой номер к карте лояльности,Для изменения привязанного номера к карте лоял...,ЛК
1705,Не открывается итоговый тест на директора мага...,"Создайте, пожалуйста, обращение в ИТ поддержк...",поддержка


In [7]:
categories = sorted(df['category'].unique())
categories # тут отпуск и Отпуск разные категории

['SED',
 'БиР',
 'ЛК',
 'МЧД',
 'Отпуск',
 'СБ',
 'ЭЦП',
 'автомобиль',
 'больничный',
 'выручай-карта',
 'график работы',
 'дмс',
 'доверенность',
 'документооборот',
 'зарплата',
 'заявки',
 'командировка',
 'материальная помощь',
 'моя карьера',
 'налоговый вычет',
 'обучение',
 'оператор',
 'отгул',
 'отпуск',
 'перевод',
 'поддержка',
 'прием на работу',
 'справка',
 'табель',
 'увольнение',
 'удаленная работа',
 'уход за больным']

In [8]:
#приведем к нижнему регистру и отбросим дубликаты

for i in df.columns:
    df[i] = df [i].str.lower()
df.drop_duplicates(inplace=True)

In [9]:
#найдем категории, в которых всего 1 запрос

grouped_df = df.groupby('category')['question'].count().sort_values()
counts = grouped_df.reset_index(name='question_count')
print(counts)

df = pd.merge(df, counts, on='category')

               category  question_count
0                   sed               1
1                    сб               1
2         выручай-карта               1
3                   мчд               1
4               перевод               2
5                   дмс               3
6          командировка               5
7              оператор               5
8              обучение               6
9          доверенность               7
10           автомобиль               8
11  материальная помощь               8
12              справка               9
13      уход за больным               9
14      налоговый вычет              12
15           больничный              17
16      документооборот              22
17        график работы              28
18                  эцп              33
19             зарплата              45
20               заявки              47
21                отгул              48
22      прием на работу              48
23                  бир              50


# Подсчет количества сочетаний

In [10]:
def count_questions_by_category_content(data):
    counts = data.groupby(['category', 'content']).size().reset_index(name='question_count')
    return pd.merge(data, counts, on=['category', 'content'])

df = count_questions_by_category_content(df)

In [11]:
df

,question,content,category,question_count_x,question_count_y
0,"я сменил автомобить, на учет еще не поставил, ...",для внесения данных по личному автомобилю обра...,автомобиль,8,8
1,не отображается автомобиль в личном кабинете.,для внесения данных по личному автомобилю обра...,автомобиль,8,8
2,добавить автомобиль,для внесения данных по личному автомобилю обра...,автомобиль,8,8
3,хочу внести данные об автомобиле,для внесения данных по личному автомобилю обра...,автомобиль,8,8
4,как внести данные об автомобиле?,для внесения данных по личному автомобилю обра...,автомобиль,8,8
...,...,...,...,...,...
1635,вернуть трек обучения,"создайте, пожалуйста, обращение в ит поддержк...",поддержка,273,4
1636,при входе на портал обучения выходит ошибка 505,данная ошибка возможна при переходе на портал\...,обучение,6,6
1637,привязать другой номер к карте лояльности,для изменения привязанного номера к карте лоял...,лк,453,1
1638,не открывается итоговый тест на директора мага...,"создайте, пожалуйста, обращение в ит поддержк...",поддержка,273,4


In [12]:
# категории до 5 значений
categories_to_augment_malo = counts[counts['question_count'] <= 5]['category'].to_list()
categories_to_augment_malo

['sed',
 'сб',
 'выручай-карта',
 'мчд',
 'перевод',
 'дмс',
 'командировка',
 'оператор']

In [13]:
# категории более 5 значений
categories_to_augment_mnogo = counts[counts['question_count'] >5]['category'].to_list()
categories_to_augment_mnogo

['обучение',
 'доверенность',
 'автомобиль',
 'материальная помощь',
 'справка',
 'уход за больным',
 'налоговый вычет',
 'больничный',
 'документооборот',
 'график работы',
 'эцп',
 'зарплата',
 'заявки',
 'отгул',
 'прием на работу',
 'бир',
 'моя карьера',
 'увольнение',
 'удаленная работа',
 'отпуск',
 'табель',
 'поддержка',
 'лк']

# Перефразирование

In [14]:
def rephrase_question(question, model):
    response = model.Completion.create(
        engine="text-davinci-003",  # https://medium.com/technology-hits/new-gpt-3-model-text-davinci-003-is-awesome-ada11ef660a9
        prompt=f"Перефразируй следующий вопрос, сохраняя смысл: '{question}'. Представь, что ты обычный человек из России и задаешь вопросы чате корпоративной поддержки.",
        max_tokens=50
    )
    return response.choices[0].text.strip()

# Аугментация датасета

In [ ]:
def augment_by_category_content(data, model):
    augmented_data = data.copy()
    
    # Уникальные сочетания category и content
    unique_combinations = data.groupby(['category', 'content'])

    for (category, content), group in unique_combinations:
        question_count = len(group)
        
        # Если вопросов <= 3, добавляем 10 перефразированных вариантов
        if question_count <= 3:
            for question in group['question']:
                for _ in range(10):
                    new_question = rephrase_question(question, model)
                    augmented_data = augmented_data.append({
                        'question': new_question,
                        'content': content,
                        'category': category,
                        'question_count': question_count
                    }, ignore_index=True)
        
        # Если вопросов > 5, добавляем 3 перефразированных варианта
        elif question_count > 5:
            for question in group['question']:
                for _ in range(3):
                    new_question = rephrase_question(question, model)
                    augmented_data = augmented_data.append({
                        'question': new_question,
                        'content': content,
                        'category': category,
                        'question_count': question_count
                    }, ignore_index=True)
        
    return augmented_data

# Модель

In [ ]:
model = OpenAI(api_key='АПИ АПИ АПИ') 

In [ ]:
data_with_counts = count_questions_by_category_content(df)

In [ ]:
data_with_counts

In [ ]:
final_data = augment_categories(df, model)

# Результат

In [ ]:
save_to_excel(final_data, file_path, sheet_name)